# CityBikes

Send a request to CityBikes for the city of your choice. 

In [8]:
import requests
import pandas as pd

def get_city_data(city):
    
  city_bike_networks = requests.get("http://api.citybik.es/v2/networks").json()

  list_of_dicts = []
  for city_bike_dict in city_bike_networks['networks']:
      new_city = city_bike_dict['location']['city']
      if new_city.lower() == city.lower():
          list_of_dicts.append(city_bike_dict)
          
  return list_of_dicts


In [10]:
print(get_city_data('Vancouver'))



[{'id': 'mobibikes', 'name': 'Mobi', 'location': {'latitude': 49.2827, 'longitude': -123.1207, 'city': 'Vancouver', 'country': 'CA'}, 'href': '/v2/networks/mobibikes', 'company': ['Vanncouver Bike Share Inc.', 'CycleHop LLC', 'City of Vancouver', 'Shaw Communications Inc.', 'Fifteen'], 'gbfs_href': 'https://vancouver-gbfs.smoove.pro/gbfs/2/gbfs.json'}]


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [11]:
def get_stations_info(city):
    
    station_dict = get_city_data(city)
    if not station_dict:
        print("Error: No bike company found for {}".format(city))
        return None

    network_address = station_dict[0]['href']
    url = "http://api.citybik.es/{}".format(network_address)
    return requests.get(url).json()['network']['stations']

Put your parsed results into a DataFrame.

In [12]:
get_stations_info('Vancouver')

[{'id': '00fa94ad698dc4a9e4d708d6fd32f294',
  'name': 'Chilco & Barclay',
  'latitude': 49.291909,
  'longitude': -123.140713,
  'timestamp': '2024-10-17T23:46:10.269196Z',
  'free_bikes': 9,
  'empty_slots': 9,
  'extra': {'uid': '0092',
   'renting': True,
   'returning': True,
   'last_updated': 1729208633,
   'slots': 18,
   'normal_bikes': 7,
   'ebikes': 2,
   'has_ebikes': True}},
 {'id': '012d3e06901cc222b1c2cf0a2ace3a29',
  'name': 'St George & Broadway',
  'latitude': 49.262321,
  'longitude': -123.09306,
  'timestamp': '2024-10-17T23:46:10.303195Z',
  'free_bikes': 0,
  'empty_slots': 14,
  'extra': {'uid': '0248',
   'renting': True,
   'returning': True,
   'last_updated': 1729208508,
   'slots': 14,
   'normal_bikes': 0,
   'ebikes': 0,
   'has_ebikes': True}},
 {'id': '029a505bd4422a1afd127987757f71a6',
  'name': 'Britannia Parking Lot',
  'latitude': 49.275882,
  'longitude': -123.071865,
  'timestamp': '2024-10-17T23:46:10.302439Z',
  'free_bikes': 2,
  'empty_slots': 

In [14]:
def get_available_stations(city="Paris"):
    '''
    Takes in the city name and returns a pandas dataframe containing information about the city
    Default city name is Paris
    '''
    station_info = get_stations_info(city)
    
    station_list = []
    for info in station_info:
        a_dict = {
            'id': info['id'],
            'Station Name': info['name'],
            'empty_slots': info['empty_slots'],
            'free_bikes': info['free_bikes'],
            'ebikes': info['extra'].get('ebikes', 0),  # Use .get() with a default value
            'normal_bikes': info['extra'].get('normal_bikes', 0),  # Add normal_bikes
            'latitude': info['latitude'],
            'longitude': info['longitude'],
            'timestamp': info['timestamp'],
            'Unique ID': info['extra']['uid'],
        }
        station_list.append(a_dict)
        
    return pd.DataFrame(station_list)

# Test the function
print(get_available_stations('Vancouver'))

              Station Name  empty_slots  free_bikes  ebikes  normal_bikes  \
0         Chilco & Barclay            9           9       2             7   
1     St George & Broadway           14           0       0             0   
2    Britannia Parking Lot           12           2       1             1   
3          Morton & Denman            6          19       0            19   
4      Thornton & National           10           4       0             4   
..                     ...          ...         ...     ...           ...   
253          7th & Heather           17           1       0             1   
254        Union & Dunlevy            2          14       0            14   
255    Richards & Helmcken           14           2       1             1   
256        Keefer & Abbott            0          26       0            26   
257    8th & Prince Edward           14           3       1             2   

      latitude   longitude                    timestamp Unique ID  
0    49

In [17]:
## save the dataframe to a csv file
get_available_stations('Vancouver').to_csv('../data/vancouver_bike_stations.csv', index=False)
